In [25]:
#Librerias
import joblib

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.preprocessing import StandardScaler # for standardization us
from sklearn.metrics import accuracy_score, classification_report

### Carga del dataframe de prueba

In [35]:
# Carga de datos
data = pd.read_csv("example_reduced.csv") #data set reducido de prueba SIN filtrar por len

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49999 entries, 0 to 49998
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   text          49999 non-null  object
 1   class         49999 non-null  int64 
 2   len_text      49999 non-null  int64 
 3   text_cleaned  49999 non-null  object
dtypes: int64(2), object(2)
memory usage: 1.5+ MB


### Division de X e y

In [40]:
X = data["text_cleaned"]
y = data["class"]

### Vectorized

In [29]:
vectorizer = joblib.load('vectorizer.pkl')

In [30]:
text_vect = vectorizer.transform(X)

### Standarized

In [31]:
standardized = joblib.load('standarizer.pkl')

In [32]:
X_scaled = standardized.transform(text_vect)

In [45]:
X_scaled.shape

(49999, 404235)

### prueba del modelo Xgboost con el dataframe de prueba

In [43]:
# Cargar el modelo desde el archivo .pkl
xgboost_model_loaded = joblib.load('xgboost_model_correct.pkl')

In [44]:
y_pred_prueba = xgboost_model_loaded.predict(X_scaled)

In [11]:
# Evaluación con etiquetas verdaderas
y_true_prueba = data['class']  # Obtener las etiquetas verdaderas

In [12]:
accuracy_prueba = accuracy_score(y_true_prueba, y_pred_prueba)

In [13]:
# Crear un diccionario de mapeo
mapping = {0 : 'Human', 1 : 'AI'}

In [20]:
y_true_prueba_datas = y_true_prueba.map(mapping)

In [15]:
import numpy as np


# Usar numpy.where para mapear
y_pred_mapeado = np.where(y_pred_prueba == 0, mapping[0], mapping[1])

print(y_pred_mapeado)


['Human' 'Human' 'Human' ... 'Human' 'Human' 'AI']


In [16]:
y_pred_prueba

array([0, 0, 0, ..., 0, 0, 1])

In [21]:
results_df = pd.DataFrame({
    'Text': X,
    'Generated by': y_true_prueba_datas,
    'Prediction': y_pred_mapeado,
    'Correct prediction': y_pred_prueba == y
})

In [22]:
results_df.head(30)

,Text,Generated by,Prediction,Correct prediction
0,idea driverless car appeal practic modern day ...,Human,Human,True
1,dear princip johnson write idea requir student...,AI,Human,False
2,dear princip talk student school john fn commu...,Human,Human,True
3,video game big impact peopl behavior peopl thi...,AI,Human,False
4,deer senat db write express perspect ongo deba...,AI,Human,False
5,say goe happi typic use say action improv situ...,Human,Human,True
6,advantag limit car usag evid passag provid hig...,AI,Human,False
7,ix tough situat know get help ox hardest thing...,Human,Human,True
8,consid finish high school earli student want s...,AI,Human,False
9,technolog break day ws use tell feel help way ...,Human,Human,True


In [19]:
print("Precisión del modelo en el dataframe de prueba:", accuracy_prueba)
print("\nInforme de clasificación para el dataframe de prueba:\n", classification_report(y_true_prueba, y_pred_prueba))

Precisión del modelo en el dataframe de prueba: 0.6155723114462289

Informe de clasificación para el dataframe de prueba:
               precision    recall  f1-score   support

           0       0.58      0.81      0.68     25112
           1       0.69      0.42      0.52     24887

    accuracy                           0.62     49999
   macro avg       0.64      0.61      0.60     49999
weighted avg       0.64      0.62      0.60     49999

